# E3DGE: Self-Supervised Geometry-Aware Encoder for Style-based 3D GAN Inversion (CVPR 2023)

Authored by Yushi Lan, Xuyi Meng, Shuai Yang, Chen Change Loy, Bo Dai

This colab notebook provides a quick demo of E3DGE with the model trained on FFHQ.

Useful links:

1. Github Repo: https://github.com/NIRVANALAN/E3DGE
2. Project Page: https://nirvanalan.github.io/projects/E3DGE/index.html
3. arXiv Link: https://arxiv.org/abs/2212.07409
4. Demo Video: https://drive.google.com/file/d/1yDkJfJOLeVlON7ZdRSnR34Ra_ikTVI0A/preview

In [1]:
# @title Clone Repo
!nvidia-smi
!git clone https://gitlab.com/NIRVANALAN/3dsdf_projection.git
%cd 3dsdf_projection

Mon Jul  3 18:53:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# @title Setup Environment
!!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install fvcore iopath

import sys
import torch
from IPython.display import HTML
from base64 import b64encode

pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

# validate pytorch installation
import torch
import pytorch3d
print(torch.cuda.is_available())

!pip install -r requirements.txt

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu113_pyt1110/download.html
True
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '3dsdf_projection/requirements.txt'


In [13]:
# @title Download Models
! python download_models.py # download pre-trained models

assets		    full_models     pretrained_models	 requirements.txt
calc_metrics.py     helper_scripts  pretrained_renderer  scripts
configs		    LICENSE	    project		 test_ae.py
download_models.py  logs	    README.md		 train_ae.py
  0% 0.00/63.7M [00:00<?, ?B/s]downloading pretrained_renderer/ffhq_vol_renderer.pt...
100% 63.7M/63.7M [00:00<00:00, 99.8MB/s]
  0% 0.00/203M [00:00<?, ?B/s]downloading full_models/ffhq1024x1024.pt...
100% 203M/203M [00:01<00:00, 150MB/s]
  0% 0.00/1.58G [00:00<?, ?B/s]downloading pretrained_models/E3DGE_Full_Runner.pt...
100% 1.58G/1.58G [00:09<00:00, 166MB/s]
  0% 0.00/203M [00:00<?, ?B/s]downloading full_models/Toonify400_1024x1024.pt...
100% 203M/203M [00:06<00:00, 33.2MB/s]
  0% 0.00/1.17G [00:00<?, ?B/s]downloading pretrained_models/Toonify/no_local_basic_trainer.pt.pt...
100% 1.17G/1.17G [00:13<00:00, 87.6MB/s]
  0% 0.00/175M [00:00<?, ?B/s]downloading pretrained_models/arcface_model_ir_se50.pth...
100% 175M/175M [00:02<00:00, 68.5MB/s]


In [45]:
# @title Start the Inversion!
! bash scripts/test/demo_view_synthesis.sh

+ checkpoints_dir=logs/CelebA-HQ/inference-check
+ eval_dataset_path=assets/colab_demo_img
+ test_dataset_path=assets/colab_demo_img
+ dataset_path=assets/colab_demo_img
+ hg_input_channel=64
+ netLocal_type=HGPIFuNetGANResidualResnetFC
+ batch_size=1
+ synthetic_batch_size=1
+ chunk=1
+ eval_batch=1
+ encoder_type=HybridGradualStyleEncoder_V2
+ ngpu=1
+ runner=E3DGE_Full_Runner
+ weight=0.01
+ ckpt_path=pretrained_models/E3DGE_Full_Runner.pt
+ checkpoints_dir=logs/view_synthesis/E3DGE_Full_Runner
+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ python test_ae.py --eval_dataset_path assets/colab_demo_img --test_dataset_path assets/colab_demo_img --dataset_path assets/colab_demo_img --checkpoints_dir logs/view_synthesis/E3DGE_Full_Runner --synthetic_batch_size 1 --eval_batch 1 --batch 1 --chunk 1 --expname ffhq1024x1024 --size 1024 --full_pipeline --w_space_style_pred --fpn_pigan_geo_layer_dim 128 --encoder_type HybridGradualStyleEncoder_V2 --val_n_sample 1 --lr 5e-4 --fg_mask

In [46]:
# @title Display the novel view inversion results


mp4_tex = open('logs/view_synthesis/E3DGE_Full_Runner/ffhq1024x1024/val/videos/443.jpg/sample_video__azim.mp4','rb').read()
data_url_tex = "data:video/mp4;base64," + b64encode(mp4_tex).decode()
mp4_geo = open('logs/view_synthesis/E3DGE_Full_Runner/ffhq1024x1024/val/videos/443.jpg/sample_depth_video__azim.mp4','rb').read()
data_url_geo = "data:video/mp4;base64," + b64encode(mp4_geo).decode()
HTML("""
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (data_url_tex, data_url_geo))

In [47]:
# @title Try Editing!
! bash scripts/test/demo_editing.sh

+ runner=E3DGE_Full_Runner
+ test_dataset_path=assets/colab_demo_img
+ eval_dataset_path=assets/colab_demo_img
+ dataset_path=assets/colab_demo_img
+ encoder_type=HybridGradualStyleEncoder_V2
+ batch_size=1
+ synthetic_batch_size=1
+ chunk=1
+ eval_batch=1
+ ckpt_path=pretrained_models/E3DGE_Full_Runner.pt
+ weight=0.01
+ checkpoints_dir=logs/editing/
+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ python test_ae.py --eval_dataset_path assets/colab_demo_img --dataset_path assets/colab_demo_img --test_dataset_path assets/colab_demo_img --checkpoints_dir logs/editing/ --synthetic_batch_size 1 --eval_batch 1 --batch 1 --chunk 1 --expname ffhq1024x1024 --size 1024 --full_pipeline --w_space_style_pred --fpn_pigan_geo_layer_dim 128 --encoder_type HybridGradualStyleEncoder_V2 --val_n_sample 1 --lr 5e-4 --fg_mask --continue_training --loadSize 256 --z_size 1.12 --freq_eval 5000 --freq_save 5000 --local_prior --random_flip --random_scale --random_trans --val_interval 2000 --full_pipe

In [48]:
# @title Display the edited view synthesis results (Gradually +smile)

mp4_tex = open('logs/editing/ffhq1024x1024/val/videos/443/Smiling/sample_video__azim.mp4','rb').read()
data_url_tex = "data:video/mp4;base64," + b64encode(mp4_tex).decode()
mp4_geo = open('logs/editing/ffhq1024x1024/val/videos/443/Smiling/sample_depth_video__azim.mp4','rb').read()
data_url_geo = "data:video/mp4;base64," + b64encode(mp4_geo).decode()
HTML("""
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (data_url_tex, data_url_geo))

In [49]:
# @title Try Editing!
! bash scripts/test/demo_toonify.sh

+ encoder_type=HybridGradualStyleEncoder_V2
+ hg_input_channel=64
+ netLocal_type=HGPIFuNetGANResidualResnetFC
+ eval_dataset_path=assets/colab_demo_img
+ test_dataset_path=assets/colab_demo_img
+ dataset_path=assets/colab_demo_img
+ batch_size=1
+ synthetic_batch_size=1
+ chunk=1
+ eval_batch=1
+ runner=AERunner
+ exp_prefix=v1
+ ckpt_path=pretrained_models/Toonify/no_local_basic_trainer.pt
+ checkpoints_dir=logs/toonify/AERunner_v1
+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ python test_ae.py --eval_dataset_path assets/colab_demo_img --dataset_path assets/colab_demo_img --test_dataset_path assets/colab_demo_img --checkpoints_dir logs/toonify/AERunner_v1 --synthetic_batch_size 1 --eval_batch 1 --batch 1 --chunk 1 --expname Toonify400_1024x1024 --size 1024 --full_pipeline --w_space_style_pred --fpn_pigan_geo_layer_dim 128 --encoder_type HybridGradualStyleEncoder_V2 --val_n_sample 1 --lr 5e-4 --fg_mask --continue_training --loadSize 256 --z_size 1.12 --freq_eval 5000 --fr

In [50]:
# @title Display the toonified view synthesis results

mp4_tex = open('logs/toonify/AERunner_v1/Toonify400_1024x1024/val/videos/443.jpg/sample_video__azim.mp4','rb').read()
data_url_tex = "data:video/mp4;base64," + b64encode(mp4_tex).decode()
mp4_geo = open('logs/toonify/AERunner_v1/Toonify400_1024x1024/val/videos/443.jpg/sample_depth_video__azim.mp4','rb').read()
data_url_geo = "data:video/mp4;base64," + b64encode(mp4_geo).decode()
HTML("""
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
<video width=250 controls>
      <source src="%s" type="video/mp4">
</video>
""" % (data_url_tex, data_url_geo))